In [1]:
def setupSpark():
  # Spark needs to run with Java 8 ... 
  !pip install -q findspark
  !apt-get install openjdk-8-jdk-headless > /dev/null
  !echo 2 | update-alternatives --config java > /dev/null
  !java -version
  import os, findspark
  os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
  # !echo JAVA_HOME=$JAVA_HOME
  !pip install -q pyspark
  findspark.init(spark_home='/usr/local/lib/python3.7/dist-packages/pyspark')
  !pyspark --version


setupSpark()

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~18.04-b10)
OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)
     |████████████████████████████████| 212.4 MB 58 kB/s 
     |████████████████████████████████| 198 kB 50.5 MB/s 
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 1.8.0_292
Branch HEAD
Compiled by user centos on 2021-05-24T04:27:48Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
    
spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

In [3]:
ratingsData = sc.textFile("UserItemData.txt")
ratingsData.collect()

['1001,9001,10',
 '1001,9002,1',
 '1001,9003,9',
 '1002,9001,3',
 '1002,9002,5',
 '1002,9003,1',
 '1002,9004,10',
 '1003,9001,2',
 '1003,9002,6',
 '1003,9003,2',
 '1003,9004,9',
 '1003,9005,10',
 '1003,9006,8',
 '1003,9007,9',
 '1004,9001,9',
 '1004,9002,2',
 '1004,9003,8',
 '1004,9004,3',
 '1004,9010,10',
 '1004,9011,9',
 '1004,9012,8',
 '1005,9001,8',
 '1005,9002,3',
 '1005,9003,7',
 '1005,9004,1',
 '1005,9010,9',
 '1005,9011,10',
 '1005,9012,9',
 '1005,9013,8',
 '1005,9014,1',
 '1005,9015,1',
 '1006,9001,7',
 '1006,9002,4',
 '1006,9003,8',
 '1006,9004,1',
 '1006,9010,7',
 '1006,9011,6',
 '1006,9012,9']

In [4]:
ratingVector=ratingsData.map(lambda l: l.split(','))\
        .map(lambda l:(int(l[0]), int(l[1]), float(l[2])))


In [5]:
ratingsDf=spark.createDataFrame(ratingVector, \
            ["user","item","rating"])


In [6]:
from pyspark.ml.recommendation import ALS
als = ALS(rank=10, maxIter=5)
model = als.fit(ratingsDf)

model.userFactors.orderBy("id").collect()

[Row(id=1001, features=[0.8478484153747559, 0.3832167983055115, 0.15860490500926971, -0.7782498598098755, -1.0143353939056396, -0.5040376782417297, -0.12485165148973465, 0.6936796307563782, 0.4882676899433136, 0.013109334744513035]),
 Row(id=1002, features=[-0.166128009557724, -0.41309496760368347, -0.2339731603860855, 0.016301289200782776, 1.1096529960632324, -0.5052518248558044, -0.26929977536201477, 1.150856375694275, -0.044176023453474045, -0.7406851649284363]),
 Row(id=1003, features=[-0.4328540861606598, 0.07017809897661209, -0.45907875895500183, -0.14135387539863586, 1.216313362121582, -0.2031458467245102, -0.45305150747299194, 0.8230860829353333, 0.07655487209558487, -0.21522045135498047]),
 Row(id=1004, features=[0.6335554718971252, 0.2770760953426361, 0.30716219544410706, -0.41263294219970703, -0.49889805912971497, -0.6532903909683228, -0.40081608295440674, 0.9961523413658142, 0.41936415433883667, -0.16365352272987366]),
 Row(id=1005, features=[0.33662691712379456, 0.19959934

In [7]:
testDf = spark.createDataFrame(   \
        [(1001, 9003),(1001,9004),(1001,9005)], \
        ["user","item"])

In [8]:
predictions=(model.transform(testDf).collect())
predictions

[Row(user=1001, item=9004, prediction=-0.628214955329895),
 Row(user=1001, item=9005, prediction=-2.988863229751587),
 Row(user=1001, item=9003, prediction=9.001949310302734)]